In [11]:
#importing the necessary packages for web scraping
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [12]:
#importing the Chrome browser extensions
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [13]:
# specify the url strings for the company's job posting website
url = 'https://www.foxcareers.com/Search/SearchResults'
driver = webdriver.Chrome()
driver.get(url)

In [14]:
# get job titles and links for each page and click the next button to go to the next page until no more
job_title=[]
job_link=[]

next_button = driver.find_element('xpath', '//*[@id="loadMoreButton"]')  
while next_button:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    job_title.extend([td.text for td in soup.findAll("a", {"class": "searchResultTitle"})])
    job_link.extend(['https://www.foxcareers.com' + td['href'] for td in soup.findAll("a", {"class": "searchResultTitle"})])
    try:
        next_button.click()
        time.sleep(1)
    except: break

In [15]:
def remove_space(s):
  s_new = s.replace('\n', '')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new

In [16]:
job_title = [remove_space(job) for job in job_title]

In [17]:
# create a dataframe that contains job titles and links for all job categories
df_title_link = pd.DataFrame(zip(job_title, job_link), columns=['JOB_TITLE', 'JOB_LINK'])

# drop the duplicates
df_title_link = df_title_link.drop_duplicates()

In [18]:
df_title_link.head()

,JOB_TITLE,JOB_LINK
0,MMJ/Reporter (R50021010),https://www.foxcareers.com/Search/JobDetail/R5...
1,"Editorial Producer, Lawrence Jones Cross Coun...",https://www.foxcareers.com/Search/JobDetail/R5...
2,"Assignment Editor, Weather News (R50021520)",https://www.foxcareers.com/Search/JobDetail/R5...
3,Production Assistant (R50021629),https://www.foxcareers.com/Search/JobDetail/R5...
4,"Associate Producer, Fox News Sunday (R50021633)",https://www.foxcareers.com/Search/JobDetail/R5...


# Extract the job qualifications and the job description

In [19]:
# retrieve job qualifications and descriptions
def job_description(title, link):
    qualifications = []
    description = []
    jobtitle = []
    joblink = []

    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    for i in range(len(link)):
        URL=link[i]
        driver.get(URL)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        s = ''
        d = ''

        text_pattern_descrp_1 = re.compile("A SNAPSHOT OF YOUR RESPONSIBILITIES", re.IGNORECASE)
        text_pattern_descrp_2 = re.compile("JOB DESCRIPTION", re.IGNORECASE)
        
        
        text_pattern_qual_1 = re.compile("WHAT YOU WILL NEED", re.IGNORECASE)
        text_pattern_qual_2 = re.compile("WHAT YOU NEED", re.IGNORECASE)
        text_pattern_qual_3 = re.compile("JOB RELATED KNOWLEDGE, SKILLS AND ABILITIES:", re.IGNORECASE)
        text_pattern_qual_4 = re.compile("ABOUT YOU", re.IGNORECASE)
        text_pattern_qual_5 = re.compile("QUALIFICATIONS", re.IGNORECASE)        
        
        #trys all of the qualification metrics
        try:
            tag = soup.find("b", text=text_pattern_qual_1).findNext("ul")
            s = s + tag.text
        except: pass
        
        try:
            tag = soup.find("b", text = text_patter_qual_2).findNext("ul")
            s = s + tag.text
        except: pass
        
        try:
            tag = soup.find("b", text = text_patter_qual_3).findNext("ul")
            s = s + tag.text
        except: pass
        
        try:
            tag = soup.find("b", text = text_patter_qual_4).findNext("ul")
            s = s + tag.text
        except: pass
        
        try:
            tag = soup.find("b", text = text_patter_qual_5).findNext("br")
            s = s + tag.text
        except: pass
        
        
               
        #trys all of the description metrics
        try:
            tag = soup.find("b", text=text_pattern_descrp_1).findNext("ul")
            d = tag.text
        except: pass
        
        try:
            tag = soup.find("b", text=text_pattern_descrp_2).findNext("ul")
            d = tag.text
        except: pass
        
        try:
            tag = soup.find("b", text=text_pattern_descrp_2).findNext("p")
            d = tag.text
        except: pass
        
        qualifications.append(s)
        description.append(d)   
        jobtitle.append(title[i])
        joblink.append(link[i])
        
    driver.quit()            
 
    return jobtitle, link, qualifications, description

In [20]:
title, link, qual, descrp = job_description(df_title_link['JOB_TITLE'].values, df_title_link['JOB_LINK'].values)
print(len(qual), len(descrp))

330 330


In [21]:
#create a dataframe from qualifications and descriptions
df = pd.DataFrame(zip(title, qual, link, descrp))
df.columns = ['TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']
df['COMPANY'] = 'FOX'
df = df.iloc[:, [4, 0, 1, 2, 3]]

In [22]:
df.head(300)

,COMPANY,TITLE,QUALIFICATIONS,LINK,DESCRIPTION
0,FOX,MMJ/Reporter (R50021010),,https://www.foxcareers.com/Search/JobDetail/R5...,"FOX 13 and FOX 13+, the FOX Owned & Operated d..."
1,FOX,"Editorial Producer, Lawrence Jones Cross Coun...",Bachelor’s degree in journalism or related fie...,https://www.foxcareers.com/Search/JobDetail/R5...,Lawrence Jones Cross Country is looking for a ...
2,FOX,"Assignment Editor, Weather News (R50021520)",Bachelor’s degree in journalism or related fie...,https://www.foxcareers.com/Search/JobDetail/R5...,"FOX Weather is looking for a highly motivated,..."
3,FOX,Production Assistant (R50021629),Bachelor’s degree in journalism or related fie...,https://www.foxcareers.com/Search/JobDetail/R5...,FOX News Channel is looking for a proactive an...
4,FOX,"Associate Producer, Fox News Sunday (R50021633)",Bachelor's degree in journalism or related fie...,https://www.foxcareers.com/Search/JobDetail/R5...,We are looking for an exceptional Associate Pr...
...,...,...,...,...,...
295,FOX,Freelance M.I.C.R.(Multi- Camera Insert Contr...,2-year technical degree or equivalent industry...,https://www.foxcareers.com/Search/JobDetail/R5...,Big Ten Network is looking for a Multi-Camera ...
296,FOX,Associate Producer (R50019821),3+ years prior experience in television/sports...,https://www.foxcareers.com/Search/JobDetail/R5...,FOX Sports is looking for a Associate Producer...
297,FOX,News Producer (R50020814),,https://www.foxcareers.com/Search/JobDetail/R5...,We are searching for a full-time newscast prod...
298,FOX,"Platform Software Development Engineer, Test ...",,https://www.foxcareers.com/Search/JobDetail/R5...,Blockchain Creative Labs (BCL) is a Fox Entert...


In [23]:
#remove the duplicated jobs
df_nodup = df.drop_duplicates()
print("There are {} jobs from FOX.".format(df_nodup.shape[0]))

# save the output file
df_nodup.to_csv('fox-jobs-cleaned-usa.csv')

There are 330 jobs from FOX.
